In [5]:
import pandas as pd
data = pd.read_excel('ECQ_FEV25.xlsx', sheet_name='Export')

In [6]:
data

,ANF,MUNICIPIO,ENDERECO_ID,TESTES_ECQ,TESTES_ECQ_OK
0,82,ANADIA,ALAAD_0001,31,22
1,82,ANADIA,ALAAD_0002,44,27
2,82,AGUA BRANCA,ALABN_0001,17,10
3,82,AGUA BRANCA,ALABN_0002,52,21
4,82,ARAPIRACA,ALAIR_0001,110,87
...,...,...,...,...,...
4347,79,SANTO AMARO DAS BROTAS,SESMB_0001,17,0
4348,79,SIRIRI,SESYR_0001,2,0
4349,79,TOBIAS BARRETO,SETBB_0001,31,17
4350,79,TOBIAS BARRETO,SETBB_0002,20,0


In [ ]:
import pymc as pm
import numpy as np
import arviz as az


# Organizar dados hierarquicamente
grouped = data.groupby(['ANF', 'MUNICIPIO'])

# Modelo
with pm.Model() as modelo_flex:
    # Hiperpriors para ANFs (Nível 3)
    anfs = data['ANF'].unique()
    n_anfs = len(anfs)
    
    mu_anf = pm.Beta("mu_anf", alpha=2, beta=2, shape=n_anfs)
    sigma_anf = pm.HalfNormal("sigma_anf", sigma=0.1, shape=n_anfs)
    
    # Mapeamento ANF -> índice
    anf_id_map = {anf: idx for idx, anf in enumerate(anfs)}
    
    # Loop por cada grupo (ANF + Município)
    theta_sites_list = []
    for (anf, municipio), group in grouped:
        # Dados do município
        n_tests = group['TESTES_ECQ'].values
        n_success = group['TESTES_ECQ_OK'].values
        idx_anf = anf_id_map[anf]
        
        # Nível 2: Município (mu_municipio ~ ANF)
        mu_municipio = pm.Beta(
            f"mu_municipio_{anf}_{municipio}", 
            alpha=mu_anf[idx_anf] * 100,  # phi fixo
            beta=(1 - mu_anf[idx_anf]) * 100
        )
        
        # Nível 1: Sites (theta_site ~ município)
        theta_site = pm.Beta(
            f"theta_site_{anf}_{municipio}",
            alpha=mu_municipio * 20,
            beta=(1 - mu_municipio) * 20,
            shape=len(n_tests)
        )
        
        # Likelihood
        pm.Binomial(
            f"obs_{anf}_{municipio}",
            n=n_tests,
            p=theta_site,
            observed=n_success
        )
    
    # Amostragem
    trace = pm.sample(1000, tune=500, chains=2, target_accept=0.9)

Initializing NUTS using jitter+adapt_diag...
c:\Users\F8058552\AppData\Local\anaconda3\envs\clusters\lib\site-packages\pytensor\tensor\rewriting\elemwise.py:1023: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(


In [ ]:
az.summary(trace, var_names=["mu_anf", "mu_municipio", "theta_site"])

In [ ]:
# i want to put the 95% credible interval of the mean of theta back on the data dataframe
# Calculate the posterior means and 95% credible intervals for each site
theta_posterior = trace.posterior.theta.mean(dim=["chain", "draw"]).values
theta_hdi = az.hdi(trace.posterior.theta, hdi_prob=0.95).theta.values

# Add results to the dataframe
data["theta_mean"] = theta_posterior
data["theta_lower"] = theta_hdi[:, 0]  
data["theta_upper"] = theta_hdi[:, 1]

# Calculate raw success rates for comparison
data["raw_rate"] = data["TESTES_ECQ_OK"] / data["TESTES_ECQ"]

# Display the first few rows with the new columns

In [ ]:
# Comparar ANFs
az.plot_forest(trace, var_names="mu_anf", combined=True)

# Detalhar um município específico
az.plot_forest(trace, var_names="mu_municipio_82_ANADIA", combined=True)

# Ver sites de um município
az.plot_forest(trace, filter_vars="like", var_names="theta_site_82_ANADIA", combined=True)